In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(api_key=api_key, model="gpt-4o")

In [5]:
# Create the coding agent.
coding_agent = AssistantAgent(
    name="CODING_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Generate the code for the task.",
)

# Create the testing agent.
testing_agent = AssistantAgent(
    name="TESTING_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Test the code for the task." \
        "If the code is good, then reply with 'STOP'",
)                                                                                                           

# Create the coding editor agent.
coding_editor_agent = AssistantAgent(
    name="CODING_EDITOR_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Edit the code for the task." \
        "If the code is good, then reply with 'STOP'",
)

In [10]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination


# Termination Condition
# termination_condiation = TextMentionTermination('STOP')
# termination_condiation = MaxMessageTermination(10)
termination_condiation = TextMentionTermination('STOP') | MaxMessageTermination(10)


# Create the group chat.
team = RoundRobinGroupChat(
    participants=[coding_agent, testing_agent, coding_editor_agent],
    termination_condition=termination_condiation
)

In [11]:
from autogen_agentchat.ui import Console

await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a simple Hello world code ?"))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a simple Hello world code ?
---------- TextMessage (CODING_AGENT) ----------
Certainly! Here is a simple Python program that prints "Hello, World!" to the console:

```python
print("Hello, World!")
```

You can run this code in any Python interpreter or IDE. It will output the string "Hello, World!" to the console.
---------- TextMessage (TESTING_AGENT) ----------
STOP


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a simple Hello world code ?', type='TextMessage'), TextMessage(source='CODING_AGENT', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=56), metadata={}, content='Certainly! Here is a simple Python program that prints "Hello, World!" to the console:\n\n```python\nprint("Hello, World!")\n```\n\nYou can run this code in any Python interpreter or IDE. It will output the string "Hello, World!" to the console.', type='TextMessage'), TextMessage(source='TESTING_AGENT', models_usage=RequestUsage(prompt_tokens=113, completion_tokens=1), metadata={}, content='STOP', type='TextMessage')], stop_reason="Text 'STOP' mentioned")

## Managing State of Team and Agent

In [14]:
agent_state = await coding_agent.save_state()
agent_state

{'type': 'AssistantAgentState',
 'version': '1.0.0',
 'llm_context': {'messages': [{'content': 'Write a simple Hello world code ?',
    'source': 'user',
    'type': 'UserMessage'},
   {'content': 'Certainly! Here is a simple Python program that prints "Hello, World!" to the console:\n\n```python\nprint("Hello, World!")\n```\n\nYou can run this code in any Python interpreter or IDE. It will output the string "Hello, World!" to the console.',
    'thought': None,
    'source': 'CODING_AGENT',
    'type': 'AssistantMessage'}]}}

In [15]:
team_state = await team.save_state()
team_state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'CODING_AGENT': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a simple Hello world code ?',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'Certainly! Here is a simple Python program that prints "Hello, World!" to the console:\n\n```python\nprint("Hello, World!")\n```\n\nYou can run this code in any Python interpreter or IDE. It will output the string "Hello, World!" to the console.',
       'thought': None,
       'source': 'CODING_AGENT',
       'type': 'AssistantMessage'}]}},
   'message_buffer': [{'source': 'TESTING_AGENT',
     'models_usage': {'prompt_tokens': 113, 'completion_tokens': 1},
     'metadata': {},
     'content': 'STOP',
     'type': 'TextMessage'}]},
  'TESTING_AGENT': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state'

In [17]:
## Initializing the new the agent with the other agent memory

new_coding_agent = AssistantAgent(
    name = "New_Coding_Agent",
    model_client= model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Generate the code for the task."
)
new_coding_agent.load_state(agent_state)

<coroutine object AssistantAgent.load_state at 0x10d652dc0>